In [1]:
import pandas as pd
import numpy as np

# Import data
file_path = 'C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Start_10,10/hasil_optimasi.csv'  # Path absolut
df = pd.read_csv(file_path)

# Data input user (misalnya ini tetap)
cf_user = {
    'G01': 0.8,
    'G04': 0.8,
    'G06': 0.8,
    'G24': 0.8,
    'G25': 0.8,
    'G31': 0.8
}

# Fungsi hitung CF
def calculate_cf(cf_value, cf_user_value):
    return cf_user_value * cf_value

# Fungsi gabung CF
def combine_cf(cf_values):
    total_cf = 1.0
    for cf in cf_values:
        total_cf *= (1 - cf)
    return 1 - total_cf

# List semua kode penyakit
kode_penyakit_list = df['kode_penyakit'].unique()

# List untuk simpan hasil semua penyakit
all_results = []

# Loop setiap penyakit
for kode_penyakit in kode_penyakit_list:
    # Ambil cf_pakar dan cf_optimasi untuk penyakit ini
    cf_pakar = df[df['kode_penyakit'] == kode_penyakit][['kode_gejala', 'cf_pakar']].set_index('kode_gejala')
    cf_optimasi = df[df['kode_penyakit'] == kode_penyakit][['kode_gejala', 'cf_optimasi']].set_index('kode_gejala')

    cf_pakar = cf_pakar.astype({'cf_pakar': float})
    cf_optimasi = cf_optimasi.astype({'cf_optimasi': float})

    # Hasil CF per gejala
    cf_results_pakar = []
    cf_results_optimasi = []

    for gejala, user_value in cf_user.items():
        if gejala in cf_pakar.index and gejala in cf_optimasi.index:
            cf_pakar_value = cf_pakar.loc[gejala, 'cf_pakar']
            cf_optimasi_value = cf_optimasi.loc[gejala, 'cf_optimasi']

            # Hitung
            cf_pakar_result = calculate_cf(cf_pakar_value, user_value)
            cf_optimasi_result = calculate_cf(cf_optimasi_value, user_value)

            cf_results_pakar.append(cf_pakar_result)
            cf_results_optimasi.append(cf_optimasi_result)

    # Gabungkan CF
    cf_final_pakar = combine_cf(cf_results_pakar) if cf_results_pakar else 0
    cf_final_optimasi = combine_cf(cf_results_optimasi) if cf_results_optimasi else 0

    all_results.append({
        'kode_penyakit': kode_penyakit,
        'cf_akhir_pakar': cf_final_pakar,
        'cf_akhir_optimasi': cf_final_optimasi,
        'selisih': cf_final_optimasi - cf_final_pakar
    })

# Buat DataFrame hasil
results_df = pd.DataFrame(all_results)

# Diagnosis penyakit (CF tertinggi)
diagnosis = results_df.loc[results_df['cf_akhir_optimasi'].idxmax()]

# Tampilkan hasil
print("\n--- Hasil Perhitungan CF untuk Semua Penyakit ---")
print(results_df)

print("\n--- Diagnosis Berdasarkan CF Tertinggi (Optimasi) ---")
print(f"Diagnosa: {diagnosis['kode_penyakit']} dengan CF: {diagnosis['cf_akhir_optimasi']:.5f}")

print("\n--- Diagnosis Berdasarkan CF Tertinggi ---")
print(f"Diagnosa: {diagnosis['kode_penyakit']} dengan CF: {diagnosis['cf_akhir_pakar']:.5f}")




--- Hasil Perhitungan CF untuk Semua Penyakit ---
  kode_penyakit  cf_akhir_pakar  cf_akhir_optimasi       selisih
0           P01        0.000000           0.960000  9.600000e-01
1           P02        0.000000           0.800000  8.000000e-01
2           P03        0.480000           0.896007  4.160068e-01
3           P04        0.480000           0.479937 -6.302895e-05
4           P05        0.640000           0.640000 -7.296017e-10
5           P06        0.870400           0.928000  5.760000e-02
6           P07        0.000000           0.800000  8.000000e-01
7           P08        0.998731           0.999793  1.062146e-03
8           P09        0.960000           0.960000  0.000000e+00

--- Diagnosis Berdasarkan CF Tertinggi (Optimasi) ---
Diagnosa: P08 dengan CF: 0.99979

--- Diagnosis Berdasarkan CF Tertinggi ---
Diagnosa: P08 dengan CF: 0.99873
